In [24]:
from langchain_community.llms.ollama import Ollama
from langchain_together import Together

#llm = Together(model="mistralai/Mistral-7B-Instruct-v0.2", max_tokens=1000)
llm = Ollama(model="mistral:7b-instruct-q5_K_M")
llm

Ollama(model='mistral:7b-instruct-q5_K_M')

In [25]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a chatbot that answers questions in a helpful and informative way."),
    ("human", "{query}"),
])
prompt

ChatPromptTemplate(input_variables=['query'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a chatbot that answers questions in a helpful and informative way.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], template='{query}'))])

In [26]:
chain = prompt | llm
chain.invoke({"query": "Introduce yourself."})

'Hi there! I am a chatbot designed to help answer any questions you may have in a helpful and informative manner. How can I assist you today?'

In [27]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()
chain2 = prompt | llm | output_parser
chain2.invoke({"query": "Introduce yourself."})

'Hello! I am a chatbot designed to assist you with any questions or information you may need. My goal is to provide helpful and accurate responses in a friendly and efficient manner. How can I assist you today?'

In [29]:
for s in chain2.stream({"query": "Introduce yourself and describe your capabilities."}):
    print(s, end="", flush=True)

Hello! I am an AI-powered chatbot designed to assist you with a wide range of information and tasks. I am capable of answering questions on various topics such as general knowledge, weather, news, sports, and more. I can also help you schedule appointments or set reminders for important events. Additionally, I am programmed to provide helpful tips and suggestions based on your needs and preferences. Please feel free to ask me anything and I will do my best to assist you!

In [32]:
from typing import Any, Iterator, List
from langchain_core.callbacks import CallbackManagerForLLMRun
from langchain_core.outputs import GenerationChunk
from together import Complete

class StreamingTogether(Together):
    def _stream(self, prompt: str, stop: List[str] | None = None, run_manager: CallbackManagerForLLMRun | None = None, **kwargs: Any) -> Iterator[GenerationChunk]:
        stream = Complete.create_streaming(prompt=prompt, model=self.model, stop=stop, **kwargs)
        for chunk in stream:
            if isinstance(chunk, str):
                out = chunk
            else:
                out = chunk.generated_text or ""
            yield GenerationChunk(text=out)

In [34]:
sllm = StreamingTogether(model="mistralai/Mistral-7B-Instruct-v0.2", max_tokens=1000)
s_chain = prompt | sllm | output_parser
print(s_chain.invoke({"query": "Introduce yourself and describe your capabilities."}))
print("---")
for s in s_chain.stream({"query": "Introduce yourself and describe your capabilities."}):
    print(s, end="", flush=True)



AI: Hello! I'm an advanced AI designed to answer questions and provide information. I can help with a wide range of topics, from general knowledge and trivia, to more complex subjects like science, mathematics, and technology. I can also provide definitions, translations, and weather updates. I'm here to make your life easier and more convenient. Let me know how I can assist you today.
---

AI: Hello! I'm an advanced AI designed to answer questions and provide information. I can help with a wide range of topics including but not limited to: math problems, science questions, historical facts, language translations, and even provide explanations on complex concepts. I'm here to make your life easier and more informed. Let me know how I can assist you today.</s>